In [419]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import json

In [420]:
data_path = "/home/gavinlouuu/coding/uom_explore/model_input/feature_matrix.csv"
param_path = "/home/gavinlouuu/coding/uom_explore/data_science/parameter.json"

with open('parameter.json','r') as file:
    params = json.load(file)

df = pd.read_csv(data_path)
# drop experiment_id column
df.drop('experiment_id', axis=1, inplace=True)
print(type(df))
df.head()

<class 'pandas.core.frame.DataFrame'>


,140,150,152,155,157,160,162,165,167,170,...,230,232,235,237,240,242,245,247,250,channel_id
0,0.289992,0.265276,0.308857,0.316474,0.301683,0.354073,0.308133,0.308787,0.253672,0.249509,...,0.351504,0.319964,0.304643,0.301340,0.315273,0.300079,0.306160,0.297775,0.302454,0
1,0.190347,0.201155,0.261942,0.259362,0.239436,0.214840,0.243046,0.197277,0.206084,0.218502,...,0.214454,0.215712,0.209748,0.206435,0.221081,0.231202,0.197481,0.207455,0.201596,1
2,0.099843,0.100285,0.123185,0.163675,0.353439,0.310796,0.265924,0.260067,0.109472,0.078298,...,0.131489,0.084056,0.108258,0.112591,0.125032,0.115088,0.111438,0.109358,0.081325,2
3,0.290731,0.266106,0.355530,0.396649,0.635214,0.552152,0.323083,0.364723,0.307880,0.288747,...,0.302109,0.309628,0.306289,0.307220,0.311936,0.309058,0.292207,0.280615,0.278412,3
4,6.039159,6.132384,6.641051,5.897604,5.541157,5.579751,5.564905,5.541405,5.909158,5.894632,...,7.065215,6.754524,6.712956,6.570870,6.562759,6.455663,6.417492,6.324318,6.198248,4


In [421]:
# Hyperparameters
# Extract parameters from the JSON object
hidden_size = params['hidden_size']
ground_truth = params['ground_truth']
num_epochs = params['num_epochs']
batch_size = params['batch_size']
learning_rate = params['learning_rate']
momentum_value = params['momentum_value']
dropout_rate = params['dropout']

input_size = len(df.columns)-1  # removing the ground truth from the number of columns counted
num_classes = df[ground_truth].nunique()

# Replace 'target' with the name of your actual target column
X = df.drop(ground_truth, axis=1)
y = df[ground_truth]

# Split into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # This makes 60%, 20%, 20%

# Initialize the StandardScaler
scaler = StandardScaler()
# scaler = MinMaxScaler(feature_range=(0,255)) # 

# Fit the scaler to the training data and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Apply the same transformation to validation and test sets
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert arrays to tensors
X_train_scaled = torch.tensor(X_train_scaled, dtype=torch.float32).unsqueeze(1)  # Shape: [batch_size, 1, num_features]
y_train = torch.tensor(y_train.to_numpy(), dtype=torch.long)  # Convert to NumPy array first
X_val_scaled = torch.tensor(X_val_scaled, dtype=torch.float32).unsqueeze(1)  # Shape: [batch_size, 1, num_features]
y_val = torch.tensor(y_val.to_numpy(), dtype=torch.long)  # Convert to NumPy array first
X_test_scaled = torch.tensor(X_test_scaled, dtype=torch.float32).unsqueeze(1)  # Shape: [batch_size, 1, num_features]
y_test = torch.tensor(y_test.to_numpy(), dtype=torch.long)  # Convert to NumPy array first

# Create datasets
train_dataset = TensorDataset(X_train_scaled, y_train)
val_dataset = TensorDataset(X_val_scaled, y_val)
test_dataset = TensorDataset(X_test_scaled, y_test)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [422]:
# # # Define the MLP model
    
# class MLPClassifier(nn.Module):
#     def __init__(self, input_size, hidden_sizes, num_classes, dropout_prob):
#         super(MLPClassifier, self).__init__()
#         self.layers = nn.ModuleList()
#         self.batch_norms = nn.ModuleList()
#         self.dropout_prob = dropout_prob
        
#         # Input layer
#         self.layers.append(nn.Linear(input_size, hidden_sizes[0]))
#         self.batch_norms.append(nn.BatchNorm1d(hidden_sizes[0]))

#         # Hidden layers
#         for i in range(len(hidden_sizes) - 1):
#             self.layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i + 1]))
#             self.batch_norms.append(nn.BatchNorm1d(hidden_sizes[i+1]))
        
#         # Output layer
#         self.layers.append(nn.Linear(hidden_sizes[-1], num_classes))
        
#         # Softmax activation for the output layer
#         self.softmax = nn.Softmax(dim=1)
    
#     def forward(self, x):
#         for i in range(len(self.layers) - 1):
#             x = torch.relu(self.batch_norms[i](self.layers[i](x)))
#             x = F.dropout(x, p=self.dropout_prob, training=self.training)
#         x = self.layers[-1](x)
#         x = self.softmax(x)
#         return x

# # Initialize the model
# model = MLPClassifier(input_size, hidden_size, num_classes, dropout_rate)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.2)

# # Function to predict the class of new data
# def predict(model, data):
#     model.eval()
#     with torch.no_grad():
#         output = model(data)
#         _, predicted_class = torch.max(output, dim=1)
#     return predicted_class

# # Function to compute the accuracy
# def calculate_accuracy(y_pred, y_true):
#     _, predicted = torch.max(y_pred, dim=1)  # Get the index of the max log-probability
#     correct = (predicted == y_true).float().sum()
#     return correct / y_true.shape[0]

# # Training and evaluation loop
# def train_and_evaluate(model, criterion, optimizer, train_loader, val_loader, epochs=10):
#     for epoch in range(epochs):
#         model.train()
#         for X_batch, y_batch in train_loader:
#             optimizer.zero_grad()
#             y_pred = model(X_batch)
#             loss = criterion(y_pred, y_batch)
#             loss.backward()
#             optimizer.step()
        
#         # scheduler.step()
#         model.eval()
#         val_loss = 0
#         val_accuracy = 0
#         with torch.no_grad():
#             for X_val, y_val in val_loader:
#                 y_val_pred = model(X_val)
#                 val_loss += criterion(y_val_pred, y_val).item()
#                 val_accuracy += calculate_accuracy(y_val_pred, y_val)
#                 _, predicted_classes = torch.max(y_val_pred, dim=1)
#                 # print(f'Predicted: {predicted_classes}, Actual: {y_val}') # actual predicted values
        
        
#         # Average the loss and accuracy over all validation batches
#         val_loss /= len(val_loader)
#         val_accuracy /= len(val_loader)
        
#         print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# # Assuming the rest of your setup (model initialization, data loaders, etc.) is already done
# # Now you would just call train_and_evaluate
# train_and_evaluate(model, criterion, optimizer, train_loader, val_loader, epochs=num_epochs)

# 1D CNN Model 

In [425]:
out_s1 = params['out_channels1']
out_s2 = params['out_channels2']
kernel_s1 = params['kernel_size1']
kernel_s2 = params['kernel_size2']

class CNN1DClassifier(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_sizes, num_classes, dropout_rate=0.5):
        super(CNN1DClassifier, self).__init__()
        
        assert len(out_channels) == len(kernel_sizes), "The length of out_channels and kernel_sizes must be the same"
        
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()  # Adding batch normalization if needed
        
        current_in_channels = in_channels
        
        for out_channel, kernel_size in zip(out_channels, kernel_sizes):
            self.convs.append(nn.Conv1d(current_in_channels, out_channel, kernel_size=kernel_size, stride=1, padding=kernel_size // 2))
            self.bns.append(nn.BatchNorm1d(out_channel))  # Optional: Add batch normalization
            current_in_channels = out_channel
        
        # Calculate the size after all convolutional and pooling layers
        conv_output_size = input_size
        for kernel_size in kernel_sizes:
            conv_output_size = (conv_output_size + 2 * (kernel_size // 2) - (kernel_size - 1) - 1) // 1 + 1
            conv_output_size = conv_output_size // 2  # After pooling
        
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(out_channels[-1] * conv_output_size, 128)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        print(f'Input shape: {x.shape}')
        for conv, bn in zip(self.convs, self.bns):
            x = self.pool(torch.relu(bn(conv(x))))
            print(f'After conv and pool: {x.shape}')
        x = x.view(x.size(0), -1)  # Flatten the tensor
        print(f'After flatten: {x.shape}')
        x = torch.relu(self.fc1(x))
        print(f'After fc1: {x.shape}')
        x = self.dropout(x)
        x = self.fc2(x)
        print(f'After fc2: {x.shape}')
        x = self.softmax(x)
        return x

# Example usage
in_channels = 1
out_channels = [128, 256, 256, 128]
kernel_sizes = [3, 3, 3, 3]
num_classes = df[ground_truth].nunique()
model = CNN1DClassifier(in_channels, out_channels, kernel_sizes, num_classes)

# Print model to verify
print(model)


# class CNN1DClassifier(nn.Module):
#     def __init__(self, in_channels, num_classes, out_channels1=out_s1, out_channels2=out_s2, kernel_size1=kernel_s1, kernel_size2=kernel_s2, dropout_rate=dropout_rate):
#         super(CNN1DClassifier, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels1, kernel_size=kernel_size1, stride=1, padding=kernel_size1 // 2)
#         self.conv2 = nn.Conv1d(in_channels=out_channels1, out_channels=out_channels2, kernel_size=kernel_size2, stride=1, padding=kernel_size2 // 2)
#         self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        
#         # Calculate the size after two pooling layers
#         conv_output_size = input_size
#         conv_output_size = (conv_output_size + 2 * (kernel_size1 // 2) - (kernel_size1 - 1) - 1) // 1 + 1
#         conv_output_size = conv_output_size // 2  # First pooling layer
#         conv_output_size = (conv_output_size + 2 * (kernel_size2 // 2) - (kernel_size2 - 1) - 1) // 1 + 1
#         conv_output_size = conv_output_size // 2  # Second pooling layer
        
#         self.fc1 = nn.Linear(out_channels2 * conv_output_size, 128)
#         self.dropout = nn.Dropout(dropout_rate)
#         self.fc2 = nn.Linear(128, num_classes)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, x):
#         # print(f'Input shape: {x.shape}')
#         x = self.pool(torch.relu(self.conv1(x)))
#         # print(f'After conv1 and pool1: {x.shape}')
#         x = self.pool(torch.relu(self.conv2(x)))
#         # print(f'After conv2 and pool2: {x.shape}')
#         x = x.view(x.size(0), -1)  # Flatten the tensor
#         # print(f'After flatten: {x.shape}')
#         x = torch.relu(self.fc1(x))
#         # print(f'After fc1: {x.shape}')
#         x = self.dropout(x)
#         x = self.fc2(x)
#         # print(f'After fc2: {x.shape}')
#         x = self.softmax(x)
#         return x

in_channels = 1  # Since each row is a single channel of features

# Example usage
model = CNN1DClassifier(in_channels = in_channels, out_channels1=32, out_channels2=64, kernel_size1=3, kernel_size2=3, num_classes = num_classes, dropout_rate=dropout_rate)

def train_single_model(model, learning_rate, epochs, train_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
    
    return model

# Train the 1D CNN model
model = CNN1DClassifier(in_channels, num_classes)
model = train_single_model(model, learning_rate, num_epochs, train_loader)

def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in data_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

# Evaluate the 1D CNN model on the validation set
val_accuracy = evaluate_model(model, val_loader)
print(f'Validation Accuracy: {val_accuracy:.4f}')

# Evaluate the 1D CNN model on the test set
test_accuracy = evaluate_model(model, test_loader)
print(f'Test Accuracy: {test_accuracy:.4f}')


Validation Accuracy: 0.6190
Test Accuracy: 0.6667


# Ensemble attempt

In [424]:
# # Function to train a single model
# def train_single_model(input_size, hidden_sizes, num_classes, dropout_prob, learning_rate, epochs, train_loader):
#     model = MLPClassifier(input_size, hidden_sizes, num_classes, dropout_prob)
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
#     for epoch in range(epochs):
#         model.train()
#         for X_batch, y_batch in train_loader:
#             optimizer.zero_grad()
#             y_pred = model(X_batch)
#             loss = criterion(y_pred, y_batch)
#             loss.backward()
#             optimizer.step()
    
#     return model

# # Train multiple models
# num_models = 10  # Number of models to train
# models = []
# for _ in range(num_models):
#     model = train_single_model(input_size, hidden_size, num_classes, dropout_rate, learning_rate, num_epochs, train_loader)
#     models.append(model)

# # Function to combine predictions from multiple models
# def ensemble_predict(models, data_loader):
#     all_predictions = []
#     all_labels = []
#     for data, labels in data_loader:
#         model_outputs = [model(data).unsqueeze(0) for model in models]
#         avg_outputs = torch.cat(model_outputs, dim=0).mean(dim=0)
#         _, predicted_class = torch.max(avg_outputs, dim=1)
#         all_predictions.append(predicted_class)
#         all_labels.append(labels)
    
#     all_predictions = torch.cat(all_predictions)
#     all_labels = torch.cat(all_labels)
    
#     return all_predictions, all_labels

# # Function to compute accuracy
# def calculate_accuracy(y_pred, y_true):
#     correct = (y_pred == y_true).float().sum()
#     return correct / y_true.shape[0]

# # Evaluate ensemble on validation set
# predicted_classes, true_classes = ensemble_predict(models, val_loader)
# accuracy = calculate_accuracy(predicted_classes, true_classes)
# print(f'Ensemble Validation Accuracy: {accuracy:.4f}')